# Weekly Challenge 03

*Original URL* https://community.alteryx.com/t5/Weekly-Challenge/Challenge-3-Running-Averages/td-p/36814 and [**My Alteryx Approach**](https://github.com/dsmdavid/Alteryx-Weekly-Challenge/tree/master/sub_Challenge%2303)

## Brief

The goal is to create 3 and 6 month running averages for the values contained in columns: u.CAGI, d.CAGI, u.IR, d.IR, u.NonIR, d.NonIR. Create the averages by horsepower (HP) Category. 


In [1]:
import pandas as pd

## Approach I want to follow:
1. Read the data.
1. Create a function to calculate X moving average.
1. Run the function as needed and combine into a single dataframe.

In [2]:
#Read the dataframe, use the Year-Month as date and add to index; MultiIndex= Date and HP category
df = pd.read_csv("./03_files/input.csv", parse_dates=[[1,2]], index_col=[0,1])
# Fill missing values with 0
df.fillna(value = 0, inplace=True) 

In [3]:
#display` data
df.head()

,,u.CAGI,d.CAGI,u.IR,d.IR,u.NonIR,d.NonIR
Year_Month,HP Category,,,,,,
2009-01-01,10 hp,218.0,1178948.0,56.0,319316.0,162.0,859632.0
2009-02-01,10 hp,200.0,1066974.0,44.0,248806.0,156.0,818168.0
2009-03-01,10 hp,272.0,1399758.0,52.0,285730.0,220.0,1114028.0
2009-04-01,10 hp,178.0,996182.0,42.0,235730.0,136.0,760452.0
2009-05-01,10 hp,200.0,1117158.0,60.0,333908.0,140.0,783250.0


In [4]:
def returnMovingAverage(dataframe, grouping, periods):
    '''Returns the rolling mean for the given number of periods for a dataframe grouped by the grouping conditions.
    Expected values:
    dataframe = dataframe
    grouping = a list of columns/grouping elements
    periods = an int for the number of months/rows to calculate the moving average'''
    
    
    #Basic test of conditions
    if type(dataframe) != pd.core.frame.DataFrame:
        raise TypeError('First argument must be a dataframe')
    elif type(grouping) != list:
        raise TypeError('Second argument must be a list of elements to group by')
    elif type(periods) != int:
        raise TypeError('Third argument must be an int --number of months to calculate the moving calculation')
        
    
    #Calculate the moving average
    df = dataframe.groupby(by=grouping).rolling(periods, min_periods = 1).mean()

    #Rename columns to add the prefix:
    new_columns = []
    for column in df.columns:
        new_columns.append(('r'+str(periods)+'mo'+'_'+column).replace('.','_'))
    df.columns = new_columns
    
    #Rearrange the index
    df.index = df.index.droplevel(level=2)
    df = df.reorder_levels([1,0])
    
    
    return df

In [5]:
list_of_df = [df]
for period in (3,6):
    list_of_df.append(returnMovingAverage(df, ['HP Category'],period))
    

## Differences with the Alteryx Solution:
The generate rows in Alteryx is taking the closest value to fill the empty/absent values. Thus, the moving average of the first row is 3\**First row value*/3; the moving average of the second row is: (2\**First row value* + *Second row value*)/3.
Here I had 2 different approaches: either return **nan** when the value is missing, or allow the calculation with a single value

In [6]:
output = pd.concat(list_of_df, axis=1)
output.head(10)

,,u.CAGI,d.CAGI,u.IR,d.IR,u.NonIR,d.NonIR,r3mo_u_CAGI,r3mo_d_CAGI,r3mo_u_IR,r3mo_d_IR,r3mo_u_NonIR,r3mo_d_NonIR,r6mo_u_CAGI,r6mo_d_CAGI,r6mo_u_IR,r6mo_d_IR,r6mo_u_NonIR,r6mo_d_NonIR
Year_Month,HP Category,,,,,,,,,,,,,,,,,,
2009-01-01,10 hp,218.0,1178948.0,56.0,319316.0,162.0,859632.0,218.000000,1.178948e+06,56.000000,319316.000000,162.000000,8.596320e+05,218.000000,1.178948e+06,56.000000,319316.000000,162.000000,8.596320e+05
2009-02-01,10 hp,200.0,1066974.0,44.0,248806.0,156.0,818168.0,209.000000,1.122961e+06,50.000000,284061.000000,159.000000,8.389000e+05,209.000000,1.122961e+06,50.000000,284061.000000,159.000000,8.389000e+05
2009-03-01,10 hp,272.0,1399758.0,52.0,285730.0,220.0,1114028.0,230.000000,1.215227e+06,50.666667,284617.333333,179.333333,9.306093e+05,230.000000,1.215227e+06,50.666667,284617.333333,179.333333,9.306093e+05
2009-04-01,10 hp,178.0,996182.0,42.0,235730.0,136.0,760452.0,216.666667,1.154305e+06,46.000000,256755.333333,170.666667,8.975493e+05,217.000000,1.160466e+06,48.500000,272395.500000,168.500000,8.880700e+05
2009-05-01,10 hp,200.0,1117158.0,60.0,333908.0,140.0,783250.0,216.666667,1.171033e+06,51.333333,285122.666667,165.333333,8.859100e+05,213.600000,1.151804e+06,50.800000,284698.000000,162.800000,8.671060e+05
2009-06-01,10 hp,258.0,1356456.0,70.0,371568.0,188.0,984888.0,212.000000,1.156599e+06,57.333333,313735.333333,154.666667,8.428633e+05,221.000000,1.185913e+06,54.000000,299176.333333,167.000000,8.867363e+05
2009-07-01,10 hp,270.0,1358426.0,68.0,343940.0,202.0,1014486.0,242.666667,1.277347e+06,66.000000,349805.333333,176.666667,9.275413e+05,229.666667,1.215826e+06,56.000000,303280.333333,173.666667,9.125453e+05
2009-08-01,10 hp,260.0,1170142.0,48.0,236706.0,212.0,933436.0,262.666667,1.295008e+06,62.000000,317404.666667,200.666667,9.776033e+05,239.666667,1.233020e+06,56.666667,301263.666667,183.000000,9.317567e+05
2009-09-01,10 hp,402.0,1876124.0,74.0,386406.0,328.0,1489718.0,310.666667,1.468231e+06,63.333333,322350.666667,247.333333,1.145880e+06,261.333333,1.312415e+06,60.333333,318043.000000,201.000000,9.943717e+05


In [7]:
output.tail(10)

,,u.CAGI,d.CAGI,u.IR,d.IR,u.NonIR,d.NonIR,r3mo_u_CAGI,r3mo_d_CAGI,r3mo_u_IR,r3mo_d_IR,r3mo_u_NonIR,r3mo_d_NonIR,r6mo_u_CAGI,r6mo_d_CAGI,r6mo_u_IR,r6mo_d_IR,r6mo_u_NonIR,r6mo_d_NonIR
Year_Month,HP Category,,,,,,,,,,,,,,,,,,
2014-03-01,75 hp,200.0,5063774.0,50.0,1236018.0,150.0,3827756.0,208.666667,5.413948e+06,52.666667,1.364643e+06,156.000000,4.049305e+06,210.333333,5.485286e+06,49.333333,1.234984e+06,161.000000,4.250302e+06
2014-04-01,75 hp,196.0,5210746.0,68.0,1890194.0,128.0,3320552.0,201.333333,5.272311e+06,58.000000,1.566264e+06,143.333333,3.706047e+06,211.000000,5.545836e+06,57.000000,1.466396e+06,154.000000,4.079441e+06
2014-05-01,75 hp,186.0,4572086.0,44.0,1114995.8,142.0,3457090.2,194.000000,4.948869e+06,54.000000,1.413736e+06,140.000000,3.535133e+06,210.000000,5.436711e+06,58.000000,1.502228e+06,152.000000,3.934483e+06
2014-06-01,75 hp,208.0,5458388.0,54.0,1346500.1,154.0,4111887.9,196.666667,5.080407e+06,55.333333,1.450563e+06,141.333333,3.629843e+06,202.666667,5.247177e+06,54.000000,1.407603e+06,148.666667,3.839574e+06
2014-07-01,75 hp,0.0,0.0,0.0,0.0,0.0,0.0,131.333333,3.343491e+06,32.666667,8.204986e+05,98.666667,2.522993e+06,166.333333,4.307901e+06,45.333333,1.193381e+06,121.000000,3.114520e+06
2014-08-01,75 hp,0.0,0.0,0.0,0.0,0.0,0.0,69.333333,1.819463e+06,18.000000,4.488334e+05,51.333333,1.370629e+06,131.666667,3.384166e+06,36.000000,9.312847e+05,95.666667,2.452881e+06
2014-09-01,75 hp,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000,7.761021e-11,0.000000,0.000000e+00,98.333333,2.540203e+06,27.666667,7.252817e+05,70.666667,1.814922e+06
2014-10-01,75 hp,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000,7.761021e-11,0.000000,0.000000e+00,65.666667,1.671746e+06,16.333333,4.102493e+05,49.333333,1.261496e+06
2014-11-01,75 hp,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000,7.761021e-11,0.000000,0.000000e+00,34.666667,9.097313e+05,9.000000,2.244167e+05,25.666667,6.853146e+05
